In [1]:
import tensorflow as tf

# Load the UNet model
model = tf.keras.models.load_model('/home/RUS_CIP/st185771/carlaone /project5/dataA/dataA/carla-image-segmentation-model.h5')


2024-05-26 18:18:07.026894: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-26 18:18:07.171574: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-26 18:18:07.171596: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-05-26 18:18:08.114559: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [2]:
import carla
import numpy as np

def get_image_from_carla():
    client = carla.Client('localhost', 2000)
    client.set_timeout(10.0)
    
    world = client.get_world()
    blueprint_library = world.get_blueprint_library()
    
    camera_bp = blueprint_library.find('sensor.camera.rgb')
    camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
    camera = world.spawn_actor(camera_bp, camera_transform)
    
    image_data = None
    
    def process_image(image):
        nonlocal image_data
        image_data = np.frombuffer(image.raw_data, dtype=np.uint8)
        image_data = image_data.reshape((image.height, image.width, 4))
        image_data = image_data[:, :, :3]  # Drop the alpha channel

    camera.listen(lambda image: process_image(image))
    
    while image_data is None:
        world.wait_for_tick()
    
    camera.destroy()
    return image_data


In [3]:
def preprocess_image(image):
    # Resize the image to the expected shape (256, 256, 3)
    image = tf.image.resize(image, (256, 256))  # Resize to 256x256
    image = image / 255.0  # Normalize the image
    return image

def get_segmentation(image):
    preprocessed_image = preprocess_image(image)
    preprocessed_image = np.expand_dims(preprocessed_image, axis=0)  # Add batch dimension
    segmentation = model.predict(preprocessed_image)
    segmentation = np.squeeze(segmentation, axis=0)  # Remove batch dimension
    segmentation = np.argmax(segmentation, axis=-1)  # Assuming the model outputs logits for each class
    return segmentation


In [4]:
import pygame

def display_images(original, segmented):
    pygame.init()
    
    width, height = original.shape[1], original.shape[0]
    window = pygame.display.set_mode((2 * width, height))
    
    original_surface = pygame.surfarray.make_surface(original.swapaxes(0, 1))
    segmented_surface = pygame.surfarray.make_surface(segmented.swapaxes(0, 1))

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        
        window.blit(original_surface, (0, 0))
        window.blit(segmented_surface, (width, 0))
        pygame.display.flip()

    pygame.quit()

if __name__ == '__main__':
    original_image = get_image_from_carla()
    segmentation = get_segmentation(original_image)
    display_images(original_image, segmentation)


pygame 2.5.2 (SDL 2.28.2, Python 3.7.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


ValueError: in user code:

    File "/home/RUS_CIP/st185771/anaconda3/envs/py37/lib/python3.7/site-packages/keras/engine/training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "/home/RUS_CIP/st185771/anaconda3/envs/py37/lib/python3.7/site-packages/keras/engine/training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/RUS_CIP/st185771/anaconda3/envs/py37/lib/python3.7/site-packages/keras/engine/training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "/home/RUS_CIP/st185771/anaconda3/envs/py37/lib/python3.7/site-packages/keras/engine/training.py", line 2079, in predict_step
        return self(x, training=False)
    File "/home/RUS_CIP/st185771/anaconda3/envs/py37/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/RUS_CIP/st18